<a href="https://colab.research.google.com/github/Jordan-Bruno/ddp-2122/blob/main/web-frameworks/building_an_api_with_fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an API with FastAPI
In this workbook we will building a relatively simple API and serving it on our local machine (localhost). People have built APIs with Python for some time, and packages such as _flask_ have been popular for doing so. However, in the last couple of years _FastAPI_ has become a very popular solution for this, and will be our chosen package. We need to begin by installing as normal (note we'll be using _pyngrok_ as before):

In [ ]:
!pip install "fastapi[all]"
!pip install pyngrok

     |████████████████████████████████| 54 kB 937 kB/s 
     |████████████████████████████████| 58 kB 3.2 MB/s 
     |████████████████████████████████| 10.9 MB 27.2 MB/s 
     |████████████████████████████████| 255 kB 52.5 MB/s 
     |████████████████████████████████| 636 kB 46.9 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 216 kB 68.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 266 kB 47.2 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 111 kB 48.0 MB/s 
     |████████████████████████████████| 344 kB 44.0 MB/s 
     |████████████████████████████████| 3.8 MB 49.2 MB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=fccc3411f8911434237685a6b117070e6adcd04477d4e4b25b6212e46fc795de
  Stored in directory: /root/

We will also add our authtoken to the system same as we did for our _flask_ tutorial. Update this string with your own authtoken. 

In [ ]:
!ngrok authtoken "AUTHTOKEN"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Next, we will make some data that can be served from our API:

In [ ]:
from fastapi import FastAPI

datastore = {1001: {"name": "Jordan Bruno", "teacher_rank": 7},
            1002: {"name": "Liping Zheng", "teacher_rank": 2}, 
            1003: {"name": "Michael Mortenson", "teacher_rank": 5}
            }

After importing the package we build a simple data dictionary with three staff members each of which have an ID, name and their overall ranking as a teacher as maintained by WMG (as of 2021/12/02). Of course, in practice we would use a proper datastore of some kind, but this will be enough to serve our purposes. Our next task is to build some basic API functionality:

In [ ]:
app = FastAPI()

@app.get("/all")
async def get_all():
	return datastore

Our code starts by declaring a variable "app" as a FastAPI object. Having done this we can link further code to this object using decorators. This is shown in the line:

\@app.get("/all")

Here we do a few things. Firstly the @app part acts as our decorator (@{variable} means we are declaring the below function as a part of the variable given). Secondly we specify our function will work for any GET request to our given endpoint (URL). Lastly we specify the path element from our endpoint (the bit that comes after our domain) that we associate with this request. In this case we will trigger this function anytime a user hits the endpoint /all. I.e. if our domain was api.jordanbruno.com this function triggers anytime someone calls https://api.jordanbruno.com/all.

Underneath our decorator we specify a function much like we have in the bootcamp previously. There is one difference in our fist line which is that we declare this to be an asynchronus function with async. This is not really necessary for this particular task but it is usually good practice. In the most simplistic terms, async just tells the computer that the function may take some time and that it can work on other tasks while waiting. See more here. Finally we have a return statement, which in this case tells the API to return all data in the datastore.

While a useful function, in practice we would more likely want users to be able to select specific data rather than everything, and we can do this by having them pass information (e.g. ids, names, etc.) in the request (the endpoint).

In [ ]:
@app.get("/ids/{id}")
async def get_by_id(id: int):
	try:
		return datastore[id] # subset the dictionary by the ID
	except:
		return "No records found" # return if key does not exist

Much of this is unchanged except we now have a changable variable ("id") which can be used to specify a particular staff member. This forms part of the endpoint (the text of the API call - shown as "{id}" in the decorator command). It also is used in the function as an expected parameter - "(id)", which we delare as an int (rather than string as it would be in the endpoint path) - and also then in the return statement to filter the database dictionary. (Note we also use try and except to deal with the possible case of someone submitting an ID which is not in our database).

In other words, a user can make a call to

https://api.jordanbruno.com/ids/1003

and expect to see the following data returned:

_name: Michael Mortenson_<br>
_teacherrank: 5_

We can build on this concept by creating slightly more complicated functions to subset the dictionary. Such as:

In [ ]:
@app.get("/names/{name}")
async def get_by_name(name):
	output = "No records found" # placeholder if no value matched
	for sid in datastore:
		if datastore[sid]['name'] == name: # match by the name value
			output = datastore[sid] # replace placeholder with record
	return output

Ultimately we have the same process here, the only change is that we need to match a value which is nested in the dictionary. We achieve this by looping through the dictionary until we can match the "name". Note that this is not necessarily the most efficient way to do this (we could use list comprehension or similar) and also we will have issues if there is more than one record with the same name (in this case only the last record will be displayed). However, it will work for our purposes.

We can put all this code together (as below). In practice we want this code to be run as a standalone Python file (outside of Jupyter). We could save from here or copy and paste into a text/code editor and save from there. 

However, given this is "just for fun" we can rull the whole think from ngrok as we'll do below. 

In [ ]:
from fastapi import FastAPI

datastore = {1001: {"name": "Jordan Bruno", "teacher_rank": 7},
			1002: {"name": "Liping Zheng", "teacher_rank": 2}, 
			1003: {"name": "Michael Mortenson", "teacher_rank": 5}
			}

app = FastAPI()

@app.get("/all")
async def get_all():
	return datastore

@app.get("/ids/{id}")
async def get_by_id(id: int):
	try:
		return datastore[id] # subset the dictionary by the ID
	except:
		return "No records found" # return if key does not exist
	
@app.get("/names/{name}")
async def get_by_name(name):
	output = "No records found" # placeholder if no value matched
	for sid in datastore:
		if datastore[sid]['name'] == name: # match by the name value
			output = datastore[sid] # replace placeholder with record
	return output

Our last job then is to create a Ngrok connection and start the _uvicorn_ web server. We'll also print out some friendly URLs for the app itself and for the documentation. Let's get everything running and we can check the results:

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url+'/all')
print('Doc URL:', ngrok_tunnel.public_url+'/docs')
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [206]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://c748-35-194-177-132.ngrok.io/all
Doc URL: http://c748-35-194-177-132.ngrok.io/docs
INFO:     81.111.0.26:0 - "GET /all HTTP/1.1" 200 OK
INFO:     81.111.0.26:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     81.111.0.26:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     81.111.0.26:0 - "GET /openapi.json HTTP/1.1" 200 OK


As you can hopefully see, we have now built the API and routed users to our "/all" endpoint. Have a play with some of the other options ... e.g. selecting users by ID or by name. As a note, for names we need to add %20 to represent spaces. E.g. "/name/Jordan Bruno" would be "/name/Jordan%20Bruno" (also note it will be case sensitive unlike normal URLs). You can also check out the docs that have been automatically created in the industry-standard Swagger/OpenAPI standard.

## Exercise
If you have time, try creating your own for the Project Selection System product canvas we created on Monday. What would be a suitable set of endpoints? 